### Remove bad images

In [17]:
from PIL import Image 
from tqdm.notebook import tqdm
import os, glob, sys
import os.path as osp
data_dir = "z:/data/sources/dp/imagecheck2"

train_path = osp.abspath(osp.join("../train"))
if train_path not in sys.path:
    sys.path.append(train_path)

In [ ]:
datafiles = glob.glob(osp.join(data_dir, "**", "*.png"), recursive=True)
for f in tqdm(datafiles, desc="cleaning data"):
    try:
        im = Image.open(f)
        im.load()
    except:
        print(f"Bad image: {f}, removing...")
        os.remove(f)

In [12]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from dataset import ImageCaptioningDataset
from transformers import GitProcessor, GitForCausalLM

processor = GitProcessor.from_pretrained("soul11zz/image-caption-desc-only")
dataset_path = "soul11zz/image-description-large"
dt_train = load_dataset(dataset_path, split="train")
train_dataset = ImageCaptioningDataset(dt_train, processor)

dt_val = load_dataset(dataset_path, split="validation")
validation_dataset = ImageCaptioningDataset(dt_val, processor)

dt_test = load_dataset(dataset_path, split="test")
test_dataset = ImageCaptioningDataset(dt_test, processor)


2023-02-16-16-57-12  WARNING    Using custom data configuration soul11zz--image-description-large-771f77004fadc40b
2023-02-16-16-57-12  WARNING    Found cached dataset parquet (C:/Users/berno/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-description-large-771f77004fadc40b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
2023-02-16-16-57-15  WARNING    Using custom data configuration soul11zz--image-description-large-771f77004fadc40b
2023-02-16-16-57-15  WARNING    Found cached dataset parquet (C:/Users/berno/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-description-large-771f77004fadc40b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
2023-02-16-16-57-17  WARNING    Using custom data configuration soul11zz--image-description-large-771f77004fadc40b
2023-02-16-16-57-17  WARNING    Found cached dataset parquet (C:/Users/berno/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-description-large-771

In [13]:

to_remove = []
for i in tqdm(range(len(test_dataset))):
  try:
    t = test_dataset[i]
    pass
  except:
    to_remove.append(i)
    print(f"Bad image: {i}, removing...")
  

  0%|          | 0/3351 [00:00<?, ?it/s]

In [14]:
to_remove = [23645, 89323]
new_dt_train = dt_train.select(
    (
        i for i in range(len(dt_train)) 
        if i not in set(to_remove)
    )
)

In [15]:
new_dt_train[23645]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=740x493>,
 'text': 'Contemporary enclosed sunroom features beadboard ceiling over gray armless sofa accented with white and gray pillows paired with West Elm Faceted Mirror Side Tables next to white shag ottoman tucked under white lacquer console table in front of glass sliding doors.'}

In [16]:
from huggingface_hub import HfApi

repo_name = f"{dataset_path}-cleaned"
api = HfApi()
api.create_repo(repo_name, private=True, exist_ok=True, repo_type="dataset")
new_dt_train.push_to_hub(repo_name, split="train")
dt_val.push_to_hub(repo_name, split="validation")
dt_test.push_to_hub(repo_name, split="test")

  0%|          | 0/5 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

...

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

2023-02-16-17-44-05  WARNING    Updating downloaded metadata with the new split.


  0%|          | 0/4 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

2023-02-16-17-45-25  WARNING    Updating downloaded metadata with the new split.
